In [84]:
import pandas as pd
import dateparser
import datetime as datetime
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from textblob import TextBlob

In [85]:
df_yt = pd.read_csv('df_yt.csv')
df_yt_scraped2 = pd.read_csv('df_yt_scraped2.csv')
df_yt_scraped3 = pd.read_csv('df_yt_scraped3.csv')
df_yt_scraped4 = pd.read_csv('df_yt_scraped4.csv')
df_yt_scraped5 = pd.read_csv('df_yt_scraped5.csv')
df_yt_scraped6 = pd.read_csv('df_yt_scraped6.csv')
df_yt_scraped7 = pd.read_csv('df_yt_scraped7.csv')
df_yt_scraped8 = pd.read_csv('df_yt_scraped8.csv')
df_yt_scraped9 = pd.read_csv('df_yt_scraped9.csv')
df_yt_scraped10 = pd.read_csv('df_yt_scraped10.csv')


In [ ]:
df_comments = pd.concat([df_yt_scraped2, df_yt_scraped3, df_yt_scraped4, df_yt_scraped5, df_yt_scraped6, df_yt_scraped7, df_yt_scraped8, df_yt_scraped9, df_yt_scraped10], ignore_index=True)

In [91]:
def cleaning_youtube_videos(df):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    try:
        df = df[df['date'] != 'Unknown'].reset_index(drop=True)
    except:
        pass

    if df['date'].notnull().all():
        df['date'] = df['date'].astype(str)

    # apply the dateparser.parse() function to the "date" column and format the dates
    df['date'] = df['date'].apply(lambda x: dateparser.parse(x).strftime('%d-%m-%Y'))


    # Calculate Name sentiment and Description sentiment
    df['Name sentiment'] = df['Name'].apply(lambda x: sia.polarity_scores(x)['compound'])
    df['Description sentiment'] = df['description'].apply(lambda x: sia.polarity_scores(x)['compound'])
    
    # Calculate Name subjectivity and Description subjectivity
    df['Name subjectivity'] = df['Name'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['Description subjectivity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
    # Drop 'Unnamed: 0' column
    df = df.drop('Unnamed: 0', axis=1)
    
    # Rename columns
    df = df.rename(columns={'views': 'Views', 
                            'duration': 'Duration', 
                            'date': 'Date', 
                            'description': 'Description', 
                            'link': 'Link',
                            'Name': 'Title',
                            'Name subjectivity':'Title subjectivity',
                            'Name sentiment':'Title sentiment'})
    
    # Reorder columns
    df = df.reindex(columns=['Title', 
                             'Title sentiment', 
                             'Title subjectivity', 
                             'Views', 
                             'Duration', 
                             'Date', 
                             'Description', 
                             'Description sentiment', 
                             'Description subjectivity', 
                             'Link'])
    
    return df

def cleaning_youtube_comments(df):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    df = df[df['Date'] != 'Unknown'].reset_index(drop=True)
    df['Comment'] = df['Comment'].astype(str)

    if df['Date'].notnull().all():
        df['Date'] = df['Date'].astype(str)

    # apply the dateparser.parse() function to the "date" column and format the dates
    df['Date'] = df['Date'].apply(lambda x: dateparser.parse(x).strftime('%d-%m-%Y'))


    # Calculate Name sentiment and Description sentiment
    df['Comment sentiment'] = df['Comment'].apply(lambda x: sia.polarity_scores(x)['compound'])

    # Calculate Name subjectivity and Description subjectivity
    df['Comment subjectivity'] = df['Comment'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
    # Drop 'Unnamed: 0' column
    df = df.drop('Unnamed: 0', axis=1)
    
    # Rename columns
    df = df.rename(columns={'link': 'Link'})

    # Reorder columns
    df = df.reindex(columns=['Username', 
                             'Comment', 
                             'Comment sentiment', 
                             'Comment subjectivity', 
                             'Likes', 
                             'Replies', 
                             'Link'])
    
    return df


In [92]:
df_yt = cleaning_youtube_videos(df_yt)

In [94]:
df_yt.to_csv('./cleaned/df_yt_videos', index=False)

In [95]:
df_yt_comments = cleaning_youtube_comments(df_comments)

,Unnamed: 0,Username,Comment,Likes,Replies,link
4893,4893,Chandan kumar gupta,What is olympiad,0,0,https://www.youtube.com/watch?v=iyb38dzetRk


In [96]:
df_yt_comments.to_csv('./cleaned/df_yt_comments', index=False)